## PyTorch Linear Regression Model to predict crop yield

We'll create a model that predicts crop yeilds for apples and oranges (target variables) by looking at the average temperature, rainfall and humidity (input variables or features) in a region. Here's the training data:

![crop.png](crop.png)


In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

yeild_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1

yeild_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2

In [3]:
import torch
import torch.nn as nn
import numpy as np


In [19]:
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')
targets = np.array([[56,70],
                    [81,101],
                    [119,133],
                    [22,37],
                    [103,119]],dtype='float32')

In [20]:
X=torch.from_numpy(inputs)
y=torch.from_numpy(targets)
X,y

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]))

In [60]:
w=torch.randn((3,2),requires_grad=True)
b=torch.randn(2,requires_grad=True)
w,b

(tensor([[ 1.8823,  1.4529],
         [-0.2704,  0.3746],
         [-0.6210, -0.1260]], requires_grad=True),
 tensor([-0.4412, -0.5116], requires_grad=True))

In [61]:
def model(X): return X@w + b

In [62]:
def mse(y_hat, y): return ((y_hat-y)**2).mean()

### Code the Gradient Descend manually

In [51]:
lr=1e-4
for i in range(0,200):
    y_hat=model(X)
    loss=mse(y_hat,y)
    loss.backward()
    if i % 10 ==0: print(loss) 
    with torch.no_grad():
        w.sub_(lr*w.grad)              
        b.sub_(lr*b.grad)
        w.grad.zero_()
        b.grad.zero_()
        

tensor(21878.0352, grad_fn=<MeanBackward1>)
tensor(576.6278, grad_fn=<MeanBackward1>)
tensor(112.7273, grad_fn=<MeanBackward1>)
tensor(35.0276, grad_fn=<MeanBackward1>)
tensor(13.6865, grad_fn=<MeanBackward1>)
tensor(7.2661, grad_fn=<MeanBackward1>)
tensor(4.9855, grad_fn=<MeanBackward1>)
tensor(3.9168, grad_fn=<MeanBackward1>)
tensor(3.2530, grad_fn=<MeanBackward1>)
tensor(2.7626, grad_fn=<MeanBackward1>)
tensor(2.3723, grad_fn=<MeanBackward1>)
tensor(2.0532, grad_fn=<MeanBackward1>)
tensor(1.7898, grad_fn=<MeanBackward1>)
tensor(1.5718, grad_fn=<MeanBackward1>)
tensor(1.3911, grad_fn=<MeanBackward1>)
tensor(1.2414, grad_fn=<MeanBackward1>)
tensor(1.1173, grad_fn=<MeanBackward1>)
tensor(1.0144, grad_fn=<MeanBackward1>)
tensor(0.9291, grad_fn=<MeanBackward1>)
tensor(0.8585, grad_fn=<MeanBackward1>)


### Use Pytorch build-in model

In [66]:
lr=1e-4
lrmodel=nn.Linear(3,2) 
criterion=nn.functional.mse_loss
for i in range(0,200):    
    y_hat=lrmodel(X)
    loss=criterion(y_hat,y)
    loss.backward()
    if i % 10 ==0: print(loss) 
    with torch.no_grad():
        w=lrmodel.weight
        b=lrmodel.bias         
        w.sub_(lr*w.grad)              
        b.sub_(lr*b.grad)
        w.grad.zero_()
        b.grad.zero_()

tensor(18982.9941, grad_fn=<MseLossBackward>)
tensor(379.3912, grad_fn=<MseLossBackward>)
tensor(74.0150, grad_fn=<MseLossBackward>)
tensor(32.3350, grad_fn=<MseLossBackward>)
tensor(19.3587, grad_fn=<MseLossBackward>)
tensor(14.1045, grad_fn=<MseLossBackward>)
tensor(11.2326, grad_fn=<MseLossBackward>)
tensor(9.2527, grad_fn=<MseLossBackward>)
tensor(7.7198, grad_fn=<MseLossBackward>)
tensor(6.4783, grad_fn=<MseLossBackward>)
tensor(5.4570, grad_fn=<MseLossBackward>)
tensor(4.6125, grad_fn=<MseLossBackward>)
tensor(3.9131, grad_fn=<MseLossBackward>)
tensor(3.3334, grad_fn=<MseLossBackward>)
tensor(2.8529, grad_fn=<MseLossBackward>)
tensor(2.4546, grad_fn=<MseLossBackward>)
tensor(2.1244, grad_fn=<MseLossBackward>)
tensor(1.8507, grad_fn=<MseLossBackward>)
tensor(1.6238, grad_fn=<MseLossBackward>)
tensor(1.4357, grad_fn=<MseLossBackward>)


### Use dataloader & mini batch

In [71]:
from torch.utils.data import TensorDataset, DataLoader
ds=TensorDataset(X,y)
display(ds[0:2])

(tensor([[73., 67., 43.],
         [91., 88., 64.]]), tensor([[ 56.,  70.],
         [ 81., 101.]]))

In [75]:
dl=DataLoader(ds,batch_size=3,shuffle=True)


[tensor([[ 69.,  96.,  70.],
         [ 91.,  88.,  64.],
         [102.,  43.,  37.]]), tensor([[103., 119.],
         [ 81., 101.],
         [ 22.,  37.]])]

In [82]:
next(iter(dl))

[tensor([[ 69.,  96.,  70.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.]]), tensor([[103., 119.],
         [119., 133.],
         [ 22.,  37.]])]

### Use optimizer in SGD training

In [89]:
lrmodel=nn.Linear(3,2) 
criterion=nn.functional.mse_loss
opt = torch.optim.SGD(lrmodel.parameters(), lr=1e-4)

for i in range(0,200):    
    for batX, baty in dl:
        y_hat=lrmodel(X)
        loss=criterion(y_hat,y)
        loss.backward()        
        opt.step()
        opt.zero_grad()
        
    if i % 10 ==0: print(loss) 


tensor(3713.3618, grad_fn=<MseLossBackward>)
tensor(43.7201, grad_fn=<MseLossBackward>)
tensor(21.0700, grad_fn=<MseLossBackward>)
tensor(13.9887, grad_fn=<MseLossBackward>)
tensor(9.7272, grad_fn=<MseLossBackward>)
tensor(6.8428, grad_fn=<MseLossBackward>)
tensor(4.8640, grad_fn=<MseLossBackward>)
tensor(3.5043, grad_fn=<MseLossBackward>)
tensor(2.5700, grad_fn=<MseLossBackward>)
tensor(1.9279, grad_fn=<MseLossBackward>)
tensor(1.4867, grad_fn=<MseLossBackward>)
tensor(1.1835, grad_fn=<MseLossBackward>)
tensor(0.9751, grad_fn=<MseLossBackward>)
tensor(0.8320, grad_fn=<MseLossBackward>)
tensor(0.7336, grad_fn=<MseLossBackward>)
tensor(0.6659, grad_fn=<MseLossBackward>)
tensor(0.6195, grad_fn=<MseLossBackward>)
tensor(0.5875, grad_fn=<MseLossBackward>)
tensor(0.5656, grad_fn=<MseLossBackward>)
tensor(0.5505, grad_fn=<MseLossBackward>)
